In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision
import torchvision.transforms as transforms

## Loading CIFAR-10 Dataset + Preprocess

In [ ]:
class OneHotEncode:
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def __call__(self, label):
        return F.one_hot(torch.tensor(label), num_classes=self.num_classes).float()

In [ ]:
train_size = 50000
val_size = 10000
test_size = 10000

torch.manual_seed(13)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

label_transform = OneHotEncode(num_classes=10)

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True, 
    download=True, 
    transform=transform,
    target_transform=label_transform
)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_subset = Subset(train_dataset, range(train_size))
train_subset, val_subset = random_split(train_dataset, [train_size-val_size, val_size])
test_subset = Subset(test_dataset, range(test_size))

In [ ]:
classes = train_dataset.classes
print(classes)

# Dataset Preview

In [ ]:
samples = {}

for image, label in train_subset:
    label = torch.argmax(label).item()
    if label not in samples:
        samples[label] = image
    if len(samples) == len(classes):
        break

max_row = 5

fig, axes = plt.subplots(int(len(samples)/max_row), max_row, figsize=[10, 6])

keys = np.sort(list(samples.keys()))
for i, label in enumerate(keys):
    class_name = classes[i]
    sample = samples[i] * 0.5 + 0.5
    sample = sample.numpy().transpose(1, 2, 0)

    ax = axes[int(i / max_row), i % max_row]

    ax.imshow(sample)

    ax.set_title(f'{class_name}, {label}')
    ax.axis('off')


fig.suptitle('Random amples')

plt.tight_layout()

plt.show()

# Converting to Batchs

In [ ]:
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=True)

# Training Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 14 * 14, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


baseline_model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=baseline_model.parameters())

In [ ]:
def accuracy(y, output):
    y_index = torch.argmax(y, dim=1)
    output_index = torch.argmax(output, dim=1)

    return torch.mean((y_index == output_index).float())

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, epochs=30, log=True):
    train_loasses = []
    trian_accuracies = []

    validation_losses = []
    validation_accuracies = []

    for epoch in range(epochs):
        batch_loss = 0
        train_acc = 0
        val_acc = 0

        model.train()

        for X_train, y_train in train_loader:
            X_train, y_train = X_train.to(device), y_train.to(device)

            y_pred = model(X_train)

            loss = criterion(y_pred, y_train)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            batch_loss += loss.item() * y_train.shape[0]

            train_acc += accuracy(y_train, y_pred)

        train_loasses.append(batch_loss / len(train_loader.dataset))
        trian_accuracies.append(train_acc / len(train_loader.dataset))

        model.eval()
        
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)

                y_pred =  model(X_val)

                loss = criterion(y_pred, y_val)

                batch_loss += loss.item() * y_val.shape[0]
                val_acc += accuracy(y_val, y_pred)

        validation_losses.append(batch_loss / len(val_loader.dataset))
        validation_accuracies.append(val_acc / len(val_loader.dataset))

        if log and (epoch+1) % 10 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Train loss: {np.round(train_loasses[-1], 3)}, Train acc: {np.round(trian_accuracies[-1], 3)}, Val loss: {np.round(validation_losses[-1], 3)}, Val acc: {np.round(validation_accuracies[-1], 3)}')

    return train_loasses, trian_accuracies, validation_losses, validation_accuracies

In [ ]:
def plot_acc(trian_accuracies, validation_accuracies, epochs):
    plt.figure(figsize=[10, 6])

    plt.plot(range(1, epochs+1), trian_accuracies, c='blue', linestyle='--', marker='o')
    plt.plot(range(1, epochs+1), validation_accuracies, c='red', linestyle='--', marker='o')

    plt.ylabel('accuracy')
    plt.xlabel('epoch')

    plt.tight_layout()

    plt.show()


def plot_loss(train_loasses, validation_losses, epochs):
    plt.figure(figsize=[10, 6])

    plt.plot(range(1, epochs+1), train_loasses, c='blue', linestyle='--', marker='o')
    plt.plot(range(1, epochs+1), validation_losses, c='red', linestyle='--', marker='o')

    plt.ylabel('loss')
    plt.xlabel('epoch')

    plt.tight_layout()

    plt.show()


def save_model(model, path):
    torch.save(model, f=path)
    print(f'model saved to {path} successfully!')

In [ ]:
epochs = 10
train_loasses, trian_accuracies, validation_losses, validation_accuracies = train(
    model=baseline_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer, 
    criterion=criterion, 
    epochs=epochs, 
    log=True
)

In [ ]:
plot_loss(train_loasses, validation_losses, epochs=epochs)
plot_acc(trian_accuracies, validation_accuracies, epochs=epochs)

In [ ]:
save_model(baseline_model, './baseline')

# Model Imporvement

In [ ]:
class ImprovedModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.relu = nn.ReLU()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5) # 3 * 32 * 32 -> 64 * 27 * 27

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) # 64 * 13 * 13 -> 64 * 11 * 11
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) # 64 * 5 * 5 -> 63 * 3 * 3

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.averagepool = nn.AvgPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 3 * 3, 64 * 3 * 3)
        self.fc2 = nn.Linear(64 * 3 * 3, 64 * 3 * 3)
        self.fc3 = nn.Linear(64 * 3 * 3, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.averagepool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


impoved_model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=impoved_model.parameters())

In [ ]:
epochs = 10
train_loasses, trian_accuracies, validation_losses, validation_accuracies = train(model=impoved_model, optimizer=optimizer, criterion=criterion, epochs=epochs, log=True)

In [ ]:
plot_loss(train_loasses, validation_losses, epochs=epochs)
plot_acc(trian_accuracies, validation_accuracies, epochs=epochs)

In [ ]:
save_model(impoved_model, './improvedmodel')

# Adding Batch Normalization

In [ ]:
class BnModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.relu = nn.ReLU()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5) # 3 * 32 * 32 -> 64 * 27 * 27
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) # 64 * 13 * 13 -> 64 * 11 * 11
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) # 64 * 5 * 5 -> 63 * 3 * 3
        self.bn3 = nn.BatchNorm2d(64)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.averagepool = nn.AvgPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 3 * 3, 64 * 3 * 3)
        self.fc2 = nn.Linear(64 * 3 * 3, 64 * 3 * 3)
        self.fc3 = nn.Linear(64 * 3 * 3, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.averagepool(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


bn_model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=bn_model.parameters())

In [ ]:
epochs = 10
train_loasses, trian_accuracies, validation_losses, validation_accuracies = train(model=bn_model, optimizer=optimizer, criterion=criterion, epochs=epochs, log=True)

In [ ]:
plot_loss(train_loasses, validation_losses, epochs=epochs)
plot_acc(trian_accuracies, validation_accuracies, epochs=epochs)

In [ ]:
save_model(bn_model, './bnmodel')

# Adding Dropout

In [ ]:
class BnModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.relu = nn.ReLU()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5) # 3 * 32 * 32 -> 64 * 27 * 27
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) # 64 * 13 * 13 -> 64 * 11 * 11
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) # 64 * 5 * 5 -> 63 * 3 * 3
        self.bn3 = nn.BatchNorm2d(64)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.averagepool = nn.AvgPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 3 * 3, 64 * 3 * 3)
        self.fc2 = nn.Linear(64 * 3 * 3, 64 * 3 * 3)
        self.fc3 = nn.Linear(64 * 3 * 3, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.averagepool(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


do_model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=do_model.parameters())

In [ ]:
epochs = 10
train_loasses, trian_accuracies, validation_losses, validation_accuracies = train(model=do_model, optimizer=optimizer, criterion=criterion, epochs=epochs, log=True)

In [ ]:
plot_loss(train_loasses, validation_losses, epochs=epochs)
plot_acc(trian_accuracies, validation_accuracies, epochs=epochs)

In [ ]:
save_model(bn_model, './domodel')

# Test and Confusion Matrix

In [ ]:
def test(model, test_loader, criterion):
    test_loss = 0
    test_accuracy = 0

    baseline_model.eval()
    
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)

            y_pred =  model(X_test)

            loss = criterion(y_pred, y_test)

            test_loss += loss.item() * y_test.shape[0]
            test_accuracy += accuracy(y_test, y_pred)

    return test_loss, test_accuracy

In [ ]:
base_loss, base_accuracy = test(model=baseline_model, test_loader=test_loader, criterion=criterion)
print('baseline model:')
print(f'loss= {np.round(base_loss, 3)}, accuracy= {np.round(base_accuracy, 3)}')

impoved_loss, impoved_accuracy = test(model=impoved_model, test_loader=test_loader, criterion=criterion)
print('impoved model:')
print(f'loss= {np.round(impoved_loss, 3)}, accuracy= {np.round(impoved_accuracy, 3)}')

bn_loss, bn_accuracy = test(model=bn_model, test_loader=test_loader, criterion=criterion)
print('batch normalization model')
print(f'loss= {np.round(bn_loss, 3)}, accuracy= {np.round(bn_accuracy, 3)}')

do_loss, do_accuracy = test(model=do_model, test_loader=test_loader, criterion=criterion)
print('drop out model')
print(f'loss= {np.round(do_loss, 3)}, accuracy= {np.round(do_accuracy, 3)}')